In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from datetime import date
from datetime import time
import datetime as dt
from sklearn.cluster import KMeans

#### Đọc dữ liệu

In [2]:
order_detail_df = pd.read_csv("data/order_processed.csv")
user_df = pd.read_csv("data/user_processed.csv")
listening_detail_df = pd.read_csv("data/new/listening_processed.csv")
# usage_df = pd.read_excel("data/voiz_data//Usage.xlsx") 

In [3]:
order_detail_df.head(3)

,OrderID (PK),Transaction Date,UserID (FK),Bill Type,Total Amount
0,6706,2020-08-16,44822,VIP,199000
1,6998,2020-08-23,47128,VIP,199000
2,6927,2020-08-21,47134,VIP,199000


In [4]:
order_detail_df["Transaction Date"]=order_detail_df["Transaction Date"].astype("datetime64")

In [5]:
order_detail_df["Transaction Date"].describe()

<ipython-input-5-0c890eeed07c>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  order_detail_df["Transaction Date"].describe()


count                    3575
unique                    163
top       2020-12-13 00:00:00
freq                       60
first     2020-07-11 00:00:00
last      2020-12-31 00:00:00
Name: Transaction Date, dtype: object

In [6]:
order_summary_df = order_detail_df.groupby("UserID (FK)").agg(
                    {'Transaction Date': ['min', 'max', 'count']
                    }
                    ).reset_index()
order_summary_df.columns = ['User_ID','min_purchase_date','max_purchase_date','order_cnt']

order_summary_df.head(3)

,User_ID,min_purchase_date,max_purchase_date,order_cnt
0,40843,2020-09-04,2020-09-25,3
1,40861,2020-09-09,2020-09-09,1
2,41003,2020-10-27,2020-10-27,1


In [7]:
user_df.head(3)

,UserID (PK),VIP hay Free,Gender,Year of Birth,Age Range,Age,Region,Operation Systems,Registration Date,Total Listerning,Total Listening Time (Min)
0,86229,free,male,NaN,No information,NaN,VN,android,2020-11-27,218,4151.3
1,48053,free,male,1995.0,25 - 34,26.0,VN,ios,2020-08-24,19,278.6
2,46340,free,male,1988.0,25 - 34,33.0,VN,website,2020-08-19,28,170.5


In [8]:
user_df["Registration Date"]=user_df["Registration Date"].astype("datetime64")

In [9]:
user_df["Registration Date"].describe()

<ipython-input-9-bc95f8c9d5e5>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  user_df["Registration Date"].describe()


count                   66146
unique                    184
top       2020-10-31 00:00:00
freq                     1219
first     2020-07-01 00:00:00
last      2020-12-31 00:00:00
Name: Registration Date, dtype: object

In [10]:
listening_detail_df.head(3)

,PlaylistID (PK),Playlist Name,Category,Sub Category,Playlist Type,Playlist Duration (min),Actual Duration (min),UserID (FK),Listening Time,Listening Date,Listening Week,Listening Datetime,Value Rank
0,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,0.1,82846,6:05:00,12/3/2020,11/30/2020,12/3/2020 6:05,High Value
1,4577,Tư Duy Tích Cực - Bạn Là Những Gì Bạn Nghĩ (Nh...,Sách nói,Hạnh phúc,vip,79.8,15.9,56112,12:49:00,10/30/2020,10/26/2020,10/30/2020 12:49,Low Value
2,4169,50 thiên thần của bạn: Giải phóng bản thân khỏ...,Sách nói,Tâm linh,vip,324.8,6.8,94362,0:17:00,12/15/2020,12/14/2020,12/15/2020 0:17,High Value


In [11]:
listening_detail_df.dtypes

PlaylistID (PK)              int64
Playlist Name               object
Category                    object
Sub Category                object
Playlist Type               object
Playlist Duration (min)    float64
Actual Duration (min)      float64
UserID (FK)                  int64
Listening Time              object
Listening Date              object
Listening Week              object
Listening Datetime          object
Value Rank                  object
dtype: object

In [12]:
# # Chuyển Ngày phát sinh lượt nghe sang kiểu dữ liệu date
# listening_detail_df["Listening Date"] = listening_detail_df["Listening Date"].map(
#                                         lambda x:date(int(x.split("-")[2]),int(x.split("-")[1]),int(x.split("-")[0])))
listening_detail_df["Listening Date"] = listening_detail_df["Listening Date"].astype("datetime64")

In [13]:
listening_detail_df["Listening Date"].describe()

<ipython-input-13-d0c3d75ad95d>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  listening_detail_df["Listening Date"].describe()


count                  750098
unique                    184
top       2020-12-27 00:00:00
freq                    10267
first     2020-07-01 00:00:00
last      2020-12-31 00:00:00
Name: Listening Date, dtype: object

### Lấy High Value Users

In [14]:
listening_detail_df_1 = listening_detail_df.copy()

In [15]:
listening_detail_df=listening_detail_df[listening_detail_df["Value Rank"] == "High Value"]

	2. Use Cases:							
	a. Content Bundle: Gộp các contents liên quan để offer cho users theo bundle							
		1. Chọn những Content Bundle của những playlist có High Content Value ở (Output 1) >> Bundle High Content Value						
		2. Offer theo từng user						
			2.1 Application					
				2.1.1. Recommend trên trang chủ: Customize dựa trên lịch sử của người dùng, nếu là KH mới dẩy all top High Value content				
				2.2.1. Improve feature mới: Thường được nghe cùng >> Recommend trong phần dưới cùng của playlist khi user search và bấm vào playlist (~cat)				
			2.2 Streamlit					
				2.2.1. Input list các userid				
				2.2.2.	Output			
					- List các playlist recommend cho từng user (sorting)			

## Lấy high value content

In [16]:
order_summary_df.head(3)

,User_ID,min_purchase_date,max_purchase_date,order_cnt
0,40843,2020-09-04,2020-09-25,3
1,40861,2020-09-09,2020-09-09,1
2,41003,2020-10-27,2020-10-27,1


In [17]:
listening_detail_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Week',
       'Listening Datetime', 'Value Rank'],
      dtype='object')

In [18]:
listen_pay_user_df = listening_detail_df.join(other=order_summary_df[["User_ID","min_purchase_date"]].set_index("User_ID"),
                                             how="inner",on="UserID (FK)")

In [19]:
listen_pay_user_df["botton_date"]=listen_pay_user_df["min_purchase_date"].map(lambda x: (x - dt.timedelta(days = 14)))

In [20]:
listen_pay_user_df["botton_date"]=listen_pay_user_df["botton_date"].astype("datetime64")

In [21]:
listen_pay_user_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Week',
       'Listening Datetime', 'Value Rank', 'min_purchase_date', 'botton_date'],
      dtype='object')

In [22]:
listen_pay_user_df["botton_date"].dtype

dtype('<M8[ns]')

In [23]:
hv_content_df = listen_pay_user_df[listen_pay_user_df["Listening Date"] > listen_pay_user_df["botton_date"]]

In [24]:
hv_content_df = hv_content_df [hv_content_df["Listening Date"]<hv_content_df["min_purchase_date"]]


In [25]:
hv_content_df= hv_content_df[['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category','Actual Duration (min)','UserID (FK)']]

In [26]:
hv_content_summary_df=hv_content_df.groupby(['PlaylistID (PK)','Playlist Name','Category', 'Sub Category','UserID (FK)']).agg(
    {'Actual Duration (min)': 'sum'}).reset_index()
hv_content_summary_df.columns = ['PlaylistID','PlaylistName','Category', 'SubCategory','UserID','TotalDuration']

In [27]:
hv_content_summary_df.head(3)

,PlaylistID,PlaylistName,Category,SubCategory,UserID,TotalDuration
0,3808,[Sách Tóm Tắt] 10 Điều Khác Biệt Giữa Kẻ Giàu ...,Sách tóm tắt,Sách tóm tắt,48532,5.8
1,3808,[Sách Tóm Tắt] 10 Điều Khác Biệt Giữa Kẻ Giàu ...,Sách tóm tắt,Sách tóm tắt,62150,1.1
2,3808,[Sách Tóm Tắt] 10 Điều Khác Biệt Giữa Kẻ Giàu ...,Sách tóm tắt,Sách tóm tắt,62277,12.3


In [28]:
hv_content_summary_df.columns

Index(['PlaylistID', 'PlaylistName', 'Category', 'SubCategory', 'UserID',
       'TotalDuration'],
      dtype='object')

In [29]:
hv_convert_content_df=hv_content_summary_df.groupby(["PlaylistID","PlaylistName",'Category', 'SubCategory']).agg({"TotalDuration":"mean"}).reset_index()
hv_convert_content_df.columns=["PlaylistID","PlaylistName",'Category', 'SubCategory',"AvgDuration"]

In [30]:
hv_convert_content_df=hv_convert_content_df.sort_values(by=["AvgDuration"],ascending = False)

In [31]:
kq=hv_convert_content_df[hv_convert_content_df["AvgDuration"]>=20]
kq.shape[0]

474

In [32]:
hv_convert_content_df.describe(include="all")

,PlaylistID,PlaylistName,Category,SubCategory,AvgDuration
count,975.000000,975,975,975,975.000000
unique,NaN,975,5,31,NaN
top,NaN,Mọi thứ đều có thể thay đổi,Sách nói,Thiếu nhi,NaN
freq,NaN,1,462,112,NaN
mean,4602.518974,NaN,NaN,NaN,27.234392
std,388.304443,NaN,NaN,NaN,30.597589
min,3808.000000,NaN,NaN,NaN,0.000000
25%,4309.500000,NaN,NaN,NaN,8.610526
50%,4610.000000,NaN,NaN,NaN,18.900000
75%,4924.000000,NaN,NaN,NaN,35.045833


In [33]:
kq.describe(include="all")

,PlaylistID,PlaylistName,Category,SubCategory,AvgDuration
count,474.000000,474,474,474,474.000000
unique,NaN,474,5,31,NaN
top,NaN,Tại sao cần đơn giản?,Sách nói,Thành công,NaN
freq,NaN,1,283,49,NaN
mean,4637.970464,NaN,NaN,NaN,46.541484
std,386.726560,NaN,NaN,NaN,34.144320
min,3808.000000,NaN,NaN,NaN,20.000000
25%,4310.750000,NaN,NaN,NaN,26.166667
50%,4707.500000,NaN,NaN,NaN,35.570000
75%,4952.750000,NaN,NaN,NaN,51.825000


In [34]:
kq.groupby("SubCategory").size()

SubCategory
Con cái                     14
Công nghệ                    4
Danh nhân                   11
Giải trí                     2
Hạnh phúc                   36
Học ngoại ngữ                2
Kinh doanh                  31
Kinh dị                      4
Kinh điển Quốc tế           25
Kinh điển Việt Nam          23
Kiến thức                   12
Kỹ năng                     44
Lãnh đạo                    15
Lịch sử                      8
Lứa đôi                     11
Marketing                   12
Ngôn tình                   18
Pháp thoại                  14
Sách tóm tắt                34
Sức khoẻ                     7
Thiếu nhi                   16
Thiền - Tĩnh tâm             1
Thành công                  49
Tin tức                      2
Trinh thám                   9
Tâm linh                    34
Tâm lý                       7
Tâm sự                       2
Văn hoá                      2
Văn học Quốc tế Hiện đại    15
Văn học VN Hiện đại         10
dtype: int64

In [131]:
kq.to_csv("data/export/hv_convert_content.xlsx",index=False)

### Basket_analysis

In [35]:
basket_df=pd.read_excel("data/export/basket_output.xlsx")

In [130]:
basket_df1

,antecedents,next playlist
0,Trở về từ xứ tuyết,"Dấu Chân Trên Cát, Muôn Kiếp Nhân Sinh - Tập 1..."
1,Dấu Chân Trên Cát,"Muôn Kiếp Nhân Sinh - Tập 1, Trở về từ xứ tuyế..."
2,Minh Triết Trong Đời Sống,"Trở về từ xứ tuyết,Dấu Chân Trên Cát,Hành Trìn..."
3,Huyền thuật và các đạo sĩ Tây Tạng,"Dấu Chân Trên Cát,Hành Trình Về Phương Đông,Mu..."
4,Hành Trình Về Phương Đông,"Muôn Kiếp Nhân Sinh - Tập 1, Trở về từ xứ tuyế..."
5,Muôn Kiếp Nhân Sinh - Tập 1,"Hành Trình Về Phương Đông, Trở về từ xứ tuyết,..."
6,Vị Tu Sĩ Bán Chiếc Ferrari,"Hành Trình Về Phương Đông,Muôn Kiếp Nhân Sinh ..."
7,Đường Mây Qua Xứ Tuyết,Muôn Kiếp Nhân Sinh - Tập 1
8,Bên Rặng Tuyết Sơn,Muôn Kiếp Nhân Sinh - Tập 1
9,Hoa Sen Trên Tuyết,Muôn Kiếp Nhân Sinh - Tập 1


In [36]:
basket_df

,antecedents,next playlist
0,Trở về từ xứ tuyết,"Dấu Chân Trên Cát, Muôn Kiếp Nhân Sinh - Tập 1..."
1,Dấu Chân Trên Cát,"Muôn Kiếp Nhân Sinh - Tập 1, Trở về từ xứ tuyế..."
2,Minh Triết Trong Đời Sống,"Trở về từ xứ tuyết,Dấu Chân Trên Cát,Hành Trìn..."
3,Huyền thuật và các đạo sĩ Tây Tạng,"Dấu Chân Trên Cát,Hành Trình Về Phương Đông,Mu..."
4,Hành Trình Về Phương Đông,"Muôn Kiếp Nhân Sinh - Tập 1, Trở về từ xứ tuyế..."
5,Muôn Kiếp Nhân Sinh - Tập 1,"Hành Trình Về Phương Đông, Trở về từ xứ tuyết,..."
6,Vị Tu Sĩ Bán Chiếc Ferrari,"Hành Trình Về Phương Đông,Muôn Kiếp Nhân Sinh ..."
7,Đường Mây Qua Xứ Tuyết,Muôn Kiếp Nhân Sinh - Tập 1
8,Bên Rặng Tuyết Sơn,Muôn Kiếp Nhân Sinh - Tập 1
9,Hoa Sen Trên Tuyết,Muôn Kiếp Nhân Sinh - Tập 1


In [37]:
hv_convert_content_df.head(3)

,PlaylistID,PlaylistName,Category,SubCategory,AvgDuration
936,5195,Hỏa Ngục,Truyện nói,Trinh thám,315.577778
246,4313,Thần Hổ,Truyện nói,Kinh điển Việt Nam,284.800000
388,4488,Ác Mộng Trong Đêm,Truyện nói,Trinh thám,205.900000


## Content Bundle
 Cho UserID --> recommend content cho user


### 1. Recommend list based on historycal playlist and basket analysis

In [38]:
listening_detail_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Week',
       'Listening Datetime', 'Value Rank'],
      dtype='object')

In [39]:
listen_free_user_df = listening_detail_df.join(other=order_summary_df[["User_ID","min_purchase_date"]].set_index("User_ID"),
                                             how="left",on="UserID (FK)")

In [40]:
listen_free_user_df = listen_free_user_df[listen_free_user_df["min_purchase_date"].isnull()]

#### Lọc danh sách free users

In [41]:
listen_free_user_df.head(1)

,PlaylistID (PK),Playlist Name,Category,Sub Category,Playlist Type,Playlist Duration (min),Actual Duration (min),UserID (FK),Listening Time,Listening Date,Listening Week,Listening Datetime,Value Rank,min_purchase_date
0,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,0.1,82846,6:05:00,2020-12-03,11/30/2020,12/3/2020 6:05,High Value,NaT


### Kiếm ra quyển sách được nghe nhiều nhất của user

In [158]:
## Function suggest together_playlists base on basket analysis

def define_hardPlaylist(user_id, listen_free_user_df):
    df = listen_free_user_df[listen_free_user_df["UserID (FK)"]==user_id]
    bottom_date = df["Listening Date"].max() -dt.timedelta(days = 14)
    playlist = df[df["Listening Date"]>bottom_date]
    playlist_summary=playlist.groupby(["UserID (FK)","PlaylistID (PK)","Playlist Name"]).agg(
    {"Actual Duration (min)":["sum"]}).reset_index()
    playlist_summary.columns=["UserID","PlaylistID","PlaylistName","TotalDuration"]
    hard_playlist= playlist_summary.loc[playlist_summary["TotalDuration"].idxmax(),["PlaylistID","PlaylistName"]]
    return hard_playlist[1]

def suggest_together_playlists(user_id,listen_free_user_df):
    playlist = define_hardPlaylist(user_id, listen_free_user_df)
    together_playlists = basket_df[basket_df["antecedents"] == playlist]["next playlist"].to_list()
    if len(together_playlists)>1:
        return (together_playlists[0].split(","))
    else:
        return (together_playlists)
        
#     return lst

def extract_high_value(x,high_value_user):
#     high_value_user = listening_df[listening_df['Value Rank'] == 'High Value']
    high_value_user_filter = high_value_user[high_value_user['UserID (FK)'] == x]
    last_listening = high_value_user_filter.groupby(by='UserID (FK)')['Listening Date'].max().reset_index()
    last_listening.columns = ['UserID (FK)','Last Listening Date']
    test = pd.merge(high_value_user_filter, last_listening, how='left', on= 'UserID (FK)')
    test['Last Listening Date'] = test['Last Listening Date'].astype('datetime64')
    test['time window'] = (test['Last Listening Date'] - test['Listening Date']).dt.days
    test = test[test['time window'] <= 14]
    test = test.groupby(by=['UserID (FK)', 'Sub Category'])['PlaylistID (PK)'].count().reset_index()
    test.columns = ['UserID (FK)', 'Sub Category', 'Listening Count']
    test = test[test['Listening Count'] == test['Listening Count'].max()]
    test = pd.merge(test, hv_convert_content_df[['PlaylistID','PlaylistName', 'Category', 'SubCategory' , 'AvgDuration']], how='left', left_on='Sub Category', right_on = 'SubCategory' )
    lst = test['PlaylistName'].tolist()
    
    return lst

def recommend_playlists(user_id, listen_free_user_df):
    lst1 = extract_high_value(user_id,listen_free_user_df)
    lst2 = suggest_together_playlists(user_id,listen_free_user_df)
    lst3 = lst1 + lst2
    newlist=[ii for n,ii in enumerate(lst3) if ii not in lst3[:n]]
    
    return ((newlist[:3]+newlist[-3:]) if len(newlist)>6 else newlist )

In [123]:
# listen_free_user_df.to_csv("data/export/listen_free_user_df.csv")
# listening_detail_df.to_csv("data/export/listening_detail_df.csv")

In [167]:
# recommend_playlists(85514,listen_free_user_df)
suggest_together_playlists(85514,listen_free_user_df)
# define_hardPlaylist(84801,listen_free_user_df)

[]

In [185]:
user_lst = listen_free_user_df[listen_free_user_df["Playlist Name"]=="Đừng Bao Giờ Từ Bỏ Ước Mơ"]["UserID (FK)"].unique()
len(user_lst)

348

In [186]:
# thử chạy list:
recomemend_lst = {}
for user in user_lst[:10]:
    recomemend_lst[user] = recommend_playlists(user,listen_free_user_df) 
recomemend_df = pd.DataFrame()
recomemend_df["User_ID"]=recomemend_lst.keys()
recomemend_df["RecommendPlaylists"]=recomemend_lst.values()

In [199]:
recomemend_df

,User_ID,RecommendPlaylists
0,85514,"(Blog Cho Tâm Hồn 1, Bí Mật Chiếc Xô Cảm Xúc, ..."
1,84801,"(Khách Hàng Là Số 1, Cải Tổ Doanh Nghiệp Trong..."
2,60682,"(Khám Phá Những Điều Phi Thường, Để thế giới b..."
3,66283,"(Khám Phá Những Điều Phi Thường, Để thế giới b..."
4,75309,"(Khám Phá Những Điều Phi Thường, Để thế giới b..."
5,71716,"(Blog Cho Tâm Hồn 1, Bí Mật Chiếc Xô Cảm Xúc, ..."
6,70528,"(Khám Phá Những Điều Phi Thường, Để thế giới b..."
7,82390,"(Tái Tạo Tổ Chức, Triết Học Cho Người Không Ch..."
8,78771,"(Thần Hổ, Truyện Ngắn Nam Cao 1 - Chí Phèo, Tr..."
9,79347,"(Dê Con Nhanh Trí, Bí Mật Một Khu Rừng, Cùng B..."


In [206]:
## Nối free users theo clustering

clustering_df = pd.read_csv("data/clustering_processed.csv")


In [207]:
clustering_df.columns

Index(['User_ID (FK)', 'Type_user', 'MainCluster_ID',
       'MainCluster_Description'],
      dtype='object')

In [211]:
free_clustering_df = clustering_df[clustering_df["Type_user"]=="free"]

In [239]:
listen_free_user_df.columns

Index(['PlaylistID (PK)', 'Playlist Name', 'Category', 'Sub Category',
       'Playlist Type', 'Playlist Duration (min)', 'Actual Duration (min)',
       'UserID (FK)', 'Listening Time', 'Listening Date', 'Listening Week',
       'Listening Datetime', 'Value Rank', 'min_purchase_date',
       'MainCluster_ID', 'MainCluster_Description'],
      dtype='object')

In [225]:
listen_free_user_df = listen_free_user_df.join(other=free_clustering_df[["User_ID (FK)",'MainCluster_ID','MainCluster_Description']].set_index("User_ID (FK)"),
                        how = "inner", on = "UserID (FK)")
listen_free_user_df.to_csv("data/export/listen_free_user_df.csv",index= False)

In [240]:
cluster = listen_free_user_df.groupby(["MainCluster_ID","MainCluster_Description"]).agg({"UserID (FK)":"count"}).reset_index()

cluster.columns=["MainCluster_ID","MainCluster_Description","#"]

In [241]:
cluster.sort_values(by=["MainCluster_ID"])

,MainCluster_ID,MainCluster_Description,#
0,4,Lost active user,51783
1,7,"Potential, Skimming Listeners",19558
2,8,"Potential, Engaging Listeners",98687
3,9,"Loyal Users, Skimming Listeners",17705
4,10,"Loyal Users, Engaging Listeners",26133


In [247]:
l =cluster[cluster["MainCluster_Description"]=="Lost active user"]["#"].tolist()
type(l),l[0]

(list, 51783)

In [261]:
users = listen_free_user_df[listen_free_user_df["MainCluster_Description"]=="Lost active user"]["UserID (FK)"].unique()

In [263]:
type(users.tolist())

list

In [264]:
users = users[:100]

In [265]:
users

array([68280, 71674, 70213, 65043, 62648, 65204, 73219, 64694, 58540,
       59898, 83576, 62938, 75572, 82465, 64465, 64109, 63973, 76319,
       84976, 80696, 64163, 83404, 76101, 67908, 68908, 83950, 69817,
       73315, 69970, 72482, 52826, 71096, 75151, 66895, 55096, 68676,
       81317, 61336, 74814, 52798, 66250, 65117, 75829, 66478, 52982,
       68411, 73896, 61127, 70355, 78043, 82252, 75516, 73302, 71322,
       75309, 64296, 67991, 60794, 82772, 80180, 73412, 80009, 72426,
       73038, 63325, 63816, 67728, 69863, 81561, 62850, 74149, 62062,
       60525, 81473, 83221, 68902, 63240, 74985, 85028, 73348, 73256,
       77407, 75712, 73190, 62275, 67207, 63528, 66231, 65004, 51526,
       50323, 81190, 74736, 63731, 55057, 69943, 63535, 66241, 64761,
       68659], dtype=int64)

In [140]:
listen_free_user_df.dtypes

PlaylistID (PK)                     int64
Playlist Name                      object
Category                           object
Sub Category                       object
Playlist Type                      object
Playlist Duration (min)           float64
Actual Duration (min)             float64
UserID (FK)                         int64
Listening Time                     object
Listening Date             datetime64[ns]
Listening Week                     object
Listening Datetime                 object
Value Rank                         object
min_purchase_date          datetime64[ns]
dtype: object

In [13]:
hv_convert_content_df = pd.read_csv("streamlit_data/basket_output.csv")

In [14]:
hv_convert_content_df = hv_convert_content_df.drop(columns = ['Unnamed: 0'],axis=1)


,antecedents,next playlist
0,Sự Tích Con Rồng Cháu Tiên,Sự Tích Thạch Sùng
1,Sự Tích Thạch Sùng,"Sự Tích Con Rồng Cháu Tiên,Nghìn Lẻ Một Đêm - ..."
2,Trở Về Từ Cõi Sáng,"Trở về từ xứ tuyết, Huyền thuật và các đạo sĩ ..."
3,Bên Rặng Tuyết Sơn,"Trở về từ xứ tuyết, Minh Triết Trong Đời Sống,..."
4,Huyền thuật và các đạo sĩ Tây Tạng,"Trở Về Từ Cõi Sáng, Trở về từ xứ tuyết,Trở Về ..."
5,Trở về từ xứ tuyết,"Bên Rặng Tuyết Sơn , Minh Triết Trong Đời Sống..."
6,Đường Mây Qua Xứ Tuyết,"Bên Rặng Tuyết Sơn ,Trở Về Từ Cõi Sáng,Huyền t..."
7,Ngọc sáng trong hoa sen,"Huyền thuật và các đạo sĩ Tây Tạng,Minh Triết ..."
8,Hoa Sen Trên Tuyết,"Trở về từ xứ tuyết, Dấu Chân Trên Cát,Trở Về T..."
9,Minh Triết Trong Đời Sống,"Trở Về Từ Cõi Sáng, Huyền thuật và các đạo sĩ ..."


In [11]:
hv_convert_content_df.to_csv("streamlit_data/hv_convert_content.csv",index=False)